In [103]:
import time
import numpy as np
import sys
import requests
import json

In [2]:
def delay_print(s):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(0.05)

In [4]:
class Pokemon:
    def __init__(self, name, types, moves, speed, health='=========='):
        self.name = name
        self.types = types
        self.moves = moves
        self.health = health
        self.bars = 10
        self.speed = speed

In [ ]:
def poke_types(Pokemon):
    pokemon_url = 'https://pokeapi.co/api/v2/pokemon/'+Pokemon.name+'/'
    pokemon_response = requests.get(pokemon_url)
    pokemon_response_json = json.loads(pokemon_response.text)
    types = []
    for i in range(len(pokemon_response_json['types'])):
        pokemon_response_json['types'][i]['type']['name']
        types.append(pokemon_response_json['types'][i]['type']['name'])
    return types

In [ ]:
def poke_weakness(Pokemon):
    for i in poke_types(Pokemon):
        type_url = 'https://pokeapi.co/api/v2/type/'+i+'/'
        type_response = requests.get(type_url)
        type_response_json = json.loads(type_response.text)
        types = []
        for j in range(len(type_response_json['damage_relations']['double_damage_from'])):
            types.append(type_response_json['damage_relations']['double_damage_from'][j]['name'])
    return types

In [ ]:
def poke_resistance(Pokemon):
    for i in poke_types(Pokemon):
        type_url = 'https://pokeapi.co/api/v2/type/'+i+'/'
        type_response = requests.get(type_url)
        type_response_json = json.loads(type_response.text)
        types = []
        for j in range(len(type_response_json['damage_relations']['half_damage_from'])):
            types.append(type_response_json['damage_relations']['half_damage_from'][j]['name'])
    return types

In [100]:
def battle(Pokemon1, Pokemon2): 
    
    print("-----POKEMONE BATTLE-----")
    print("Pokemon 1:", Pokemon1.name)
    print("TYPE/", Pokemon1.types)
    print("\nVS")
    print("Pokemon 2:", Pokemon2.name)
    print("TYPE/", Pokemon2.types)
    
#evaluate speed of pokemon
    
    while Pokemon1.speed > Pokemon2.speed == True:
        quicker_mon = Pokemon1
        slower_mon = Pokemon2
    
    else: 
        slower_mon = Pokemon1
        quicker_mon = Pokemon2
        
    while (quicker_mon.bars > 0) and (slower_mon.bars > 0):
            
            print("Go", quicker_mon.name, "!")
            for i, x in enumerate(quicker_mon.moves):
                print(i+1, x)
            index = int(input('Pick a move: '))
            print(quicker_mon.name ,"used", list(quicker_mon.moves.keys())[index-1])
            time.sleep(1)
            print(quicker_mon.name, "did", list(quicker_mon.moves.values())[index-1], "damage!")

            slower_mon.bars -= list(quicker_mon.moves.values())[index-1]
            slower_mon.health = ""

            for i in range(int(slower_mon.bars)):
                slower_mon.health += "="
                
            print(quicker_mon.name ,"health:", quicker_mon.health)
            print(slower_mon.name ,"health:", slower_mon.health)
            
            if slower_mon.bars <= 0:
                delay_print("\n..." + slower_mon.name + ' fainted.')
                break
                
 #mirroring the attack pattern of the quicker mon but for the slower mon

            print("Go", slower_mon.name, "!")
            for i, x in enumerate(slower_mon.moves):
                print(i+1, x)
            index = int(input('Pick a move: '))
            print(slower_mon.name ,"used", list(slower_mon.moves.keys())[index-1])
            time.sleep(1)
            print(slower_mon.name, "did", list(slower_mon.moves.values())[index-1], "damage!")

            quicker_mon.bars -= list(slower_mon.moves.values())[index-1]
            quicker_mon.health = ""

            for i in range(int(quicker_mon.bars)):
                quicker_mon.health += "="
                
            print(slower_mon.name ,"health:", slower_mon.health)
            print(quicker_mon.name ,"health:", quicker_mon.health)

#battle ends here            
            
            if quicker_mon.bars <= 0:
                delay_print("\n..." + quicker_mon.name + ' fainted.')
                break

In [226]:
if __name__ == '__main__':
    gyarados = Pokemon('gyarados', 'water', {'crunch': 1, 'surf': 3}, 4)
    aerodactyl = Pokemon('aerodactyl', 'rock', {'air slash': 2, 'head smash': 3}, 7)
    
#could use api calls for move type and move power instead of using dictionaries
    #for damage calcs, could use power % 10 to simplify health ga

In [227]:
battle(gyarados, aerodactyl)

-----POKEMONE BATTLE-----
Pokemon 1: gyarados
TYPE/ water

VS
Pokemon 2: aerodactyl
TYPE/ rock
Go aerodactyl !
1 air slash
2 head smash
Pick a move: 2
aerodactyl used head smash
aerodactyl did 3 damage!
aerodactyl health: ==========
gyarados health: =======
Go gyarados !
1 crunch
2 surf
Pick a move: 1
gyarados used crunch
gyarados did 1 damage!
gyarados health: =======
aerodactyl health: =========
Go aerodactyl !
1 air slash
2 head smash
Pick a move: 2
aerodactyl used head smash
aerodactyl did 3 damage!
aerodactyl health: =========
gyarados health: ====
Go gyarados !
1 crunch
2 surf
Pick a move: 1
gyarados used crunch
gyarados did 1 damage!
gyarados health: ====
aerodactyl health: ========
Go aerodactyl !
1 air slash
2 head smash
Pick a move: 2
aerodactyl used head smash
aerodactyl did 3 damage!
aerodactyl health: ========
gyarados health: =
Go gyarados !
1 crunch
2 surf
Pick a move: 1
gyarados used crunch
gyarados did 1 damage!
gyarados health: =
aerodactyl health: =======
Go aerodact

In [ ]:
url = 'https://pokeapi.co/api/v2/move/air-cutter/'
response = requests.get(url)
response_json = json.loads(response.text)

response_json['power']
response_json['type']['name']

In [126]:
pokemon_url = 'https://pokeapi.co/api/v2/pokemon/gyarados/'
pokemon_response = requests.get(pokemon_url)
pokemon_response_json = json.loads(pokemon_response.text)

In [138]:
pokemon_response_json['types'][0]['type']['name']

'water'

In [166]:
pokemon_response_json['types'][1]['type']['name']

'flying'

In [206]:
types = []
types.append(pokemon_response_json['types'][1]['type']['name'])

In [207]:
types

['flying']

In [218]:
def poke_resistance(Pokemon):
    for i in poke_types(Pokemon):
        type_url = 'https://pokeapi.co/api/v2/type/'+i+'/'
        type_response = requests.get(type_url)
        type_response_json = json.loads(type_response.text)
        types = []
        for j in range(len(type_response_json['damage_relations']['half_damage_from'])):
            types.append(type_response_json['damage_relations']['half_damage_from'][j]['name'])
    return types

In [219]:
poke_resistance(gyarados)

['fighting', 'bug', 'grass']

In [214]:
range(len(pokemon_response_json['types']))

[]

In [163]:
type_url = 'https://pokeapi.co/api/v2/type/water/'
type_response = requests.get(type_url)
type_response_json = json.loads(type_response.text)

for i in range(len(type_response_json['damage_relations']['double_damage_from'])):
    print(type_response_json['damage_relations']['double_damage_from'][i]['name'])

grass
electric
